### They are all equally spaced

In [1]:
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from qutip import *
from matplotlib.animation import FuncAnimation
import os

In [2]:
save_path = r'C:\Users\leopo\OneDrive - UT Cloud\Uni\Semester_8\BA_mit_Git\BA_Plots\Qutip'
os.makedirs(os.path.dirname(save_path), exist_ok=True)

In [3]:
########################################                 Define constants                   #############################################
N = 50
fixed_lam   = 1
fixed_gamma = 1

fixed_dist  = 0.3 * fixed_lam
fixed_phi   = 0

x_0      = 0.2
x_max    = 0.6
x_size   = 1

phi_0    = 0
phi_max  = 2 * np.pi / N
phi_size = 50

t_max    = 4000
t_size   = 100
########################################################################################################################    
times  = np.linspace(0, t_max, t_size) * fixed_gamma
dists  = np.linspace(x_0, x_max, x_size) * fixed_lam
phis   = np.linspace(phi_0, phi_max, phi_size)

In [4]:
########################################               define the geometry                 #############################################
def dipole_vector(phi = fixed_phi):
    """Returns the dipole vector given its orientation by angle phi."""
    dipole = np.array([np.cos(phi), np.sin(phi), 0])
    return dipole

d_z = np.array([0,0,1])
d_hats = np.array([d_z for _ in range(N)])
#d_hats = np.array([dipole_vector() for _ in range(N)])

def z_rotation(angle):
    return np.array([
        [np.cos(angle), -np.sin(angle), 0],
        [np.sin(angle), np.cos(angle), 0],
        [0, 0, 1]
    ])

def ring_positions(distance = fixed_dist, N = N):
    Pos = np.zeros((N, 3))
    mu = 2 * np.pi / N
    radius = distance / 2 / np.sin(np.pi / N) # TODO something doesnt work
    helper = np.array([radius, 0, 0])
    for i in range(N):
        rotation_matrix = np.linalg.matrix_power(z_rotation(mu), i)
        Pos[i] = np.matmul(rotation_matrix, helper)
    return Pos

positions = ring_positions()
#print(positions)

# Find the highest x_value
max_x = np.max(positions[:, 0])
scale = 0.1 * max_x
# Plot the atom positions
fig, ax = plt.subplots(figsize=(15, 8))
ax.plot(positions[:, 0], positions[:, 1], 'o', markersize=10, label='atoms')
for i, position in enumerate(positions):
    ax.arrow(position[0], position[1], d_hats[i][0] * scale, d_hats[i][1] * scale, 
             head_width=0.2*scale, head_length=0.2*scale, fc='r', ec='r')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.grid(True)
ax.legend()
ax.axis('equal')
plt.show()

In [5]:
########################################             create the Hamiltonean                 #################################################
def Green_tensor(r_a, r_b, k_a):
    r_ab = r_b - r_a
    abs_r_ab = np.linalg.norm(r_ab)
    kappa = k_a * abs_r_ab
    return (np.exp(1j * kappa) / (4 * np.pi * kappa ** 2 * abs_r_ab)
                    * ((kappa ** 2 + 1j * kappa - 1) * np.eye(3)
                       + (- kappa ** 2 - 3 * 1j * kappa + 3)
                       * np.outer(r_ab, r_ab) / (abs_r_ab ** 2)))

def Gamma_matrix(distance, dipoles, lam, gamma):
    positions = ring_positions(distance)
    G_matrix = np.zeros((N, N), dtype=complex)
    for a in range(N):
        for b in range(N):
            G_matrix[a, b] = gamma
            r_a, r_b = positions[a], positions[b]
            if np.linalg.norm(r_b - r_a) > 1e-5:
                d_a, d_b = dipoles[a], dipoles[b]
                k_a = 2 * np.pi / lam
                result = (6 * np.pi * gamma / k_a * np.matmul(np.conj(d_a), np.matmul(Green_tensor(r_a, r_b, k_a), d_b.T)))
                G_matrix[a, b] = np.imag(result)
    return G_matrix

def V_matrix(distance, dipoles, lam, gamma):
    positions = ring_positions(distance, N)
    V_matrix = np.zeros((N, N), dtype=complex)
    for a in range(N):
        for b in range(N):
            r_a, r_b = positions[a], positions[b]
            V_matrix[a, b] = 0
            if np.linalg.norm(r_b - r_a) > 1e-5:
                d_a, d_b = dipoles[a], dipoles[b]
                k_a = 2 * np.pi / lam
                result = (3 * np.pi * gamma / k_a * np.matmul(np.conj(d_a), np.matmul(Green_tensor(r_a, r_b, k_a), d_b.T)))
                V_matrix[a, b] = np.real(result)
    return V_matrix
    
def H_eff(distance = fixed_dist, dipoles = [dipole_vector() for _ in range(N)], lam=fixed_lam, gamma=fixed_gamma):
    G = Gamma_matrix(distance, dipoles, lam, gamma)
    V = V_matrix(distance, dipoles, lam, gamma)
    return Qobj(V) - 1j / 2 * Qobj(G)

In [8]:
k_range = np.arange(-N//2, N//2)
def p(k):
    return 2 * np.pi / (N * fixed_dist) * k

In [9]:
def k_base(k):
    for i in range(N):
        k_hat =+ np.exp(1j * 2 * np.pi / N * (i - 1) * (k - N//2)) * basis(N,i) / np.sqrt(N)
    return k_hat

sigma = np.pi / (16 * fixed_dist)
for k in k_range:
    psi_0 =* 1 / np.sqrt(np.sqrt(2*np.pi) sigma) * np.exp(-(p(k)-p())**2 / (2 * sigma**2))

In [12]:
# Define the position indices with periodic boundary conditions
center_index = N
k_s = np.pi / fixed_dist
sigma = 0.1 * k_s
x_j = (np.arange(N) - center_index) * fixed_dist
x_j = np.mod(x_j + N//2 * fixed_dist, N * fixed_dist) - N//2 * fixed_dist
# Calculate the Gaussian wave packet in real space with periodic boundary conditions
coefficients = np.sqrt(sigma / np.sqrt(2 * np.pi)) * np.exp(-1j * k_s * x_j) * np.exp(-sigma**2 * x_j**2)
wave_packet = sum(coeff * basis(N, (j+center_index)%N) for j, coeff in enumerate(coefficients))
psi0 = wave_packet.unit()

# Plot the real and imaginary parts of the wave packet coefficients
plt.figure(figsize=(10, 6))
plt.plot(x_j, coefficients.real, 'bo-', label='Real Part')
plt.plot(x_j, coefficients.imag, 'ro-', label='Imaginary Part')
plt.xlabel('Position $x_j$')
plt.ylabel('Coefficient')
plt.title('Gaussian Wave Packet in Real Space with Periodic Boundary Conditions')
plt.legend()
plt.grid(True)
plt.show()

Psi_01 = (basis(N,0)-basis(N,1)).unit()
print(psi0)

In [32]:
from scipy.optimize import minimize
def survival_probabilities(distance, dipoles, times, gamma=fixed_gamma, Psi_0=basis(N, N//2)):
    H = H_eff(distance=distance, dipoles=dipoles, lam=fixed_lam, gamma=gamma)
    coeffs_sq_mods = np.zeros((len(times), N))
    P_surs = np.zeros((len(times)))
    
    for t_idx, t in enumerate(times):
        U = (-1j * H * t).expm()
        Psi_t = (U * Psi_0).full().flatten()
        Probs = np.abs(Psi_t)**2

        coeffs_sq_mods[t_idx, :] = Probs
        P_surs[t_idx] = Probs.sum()

    return coeffs_sq_mods, P_surs

In [33]:
distance = 0.234 * fixed_lam
dipoles = np.array([[0,0,1] for _ in range(N)])
y1, y2 = survival_probabilities(distance, dipoles, times, Psi_0=psi0)

In [38]:
# Plot atoms with their dipole moments
positions = ring_positions(fixed_dist, N)
max_x = np.max(positions[:, 0])
scale = 0.1 * max_x
optimized_dipoles = [dipole_vector(phi) for phi in  optimized_params[:-1]]
plt.figure(figsize=(8, 8))
plt.scatter(positions[:, 0], positions[:, 1], color='blue', s=100, label='Atoms')
for i, position in enumerate(positions):
    plt.arrow(position[0], position[1], dipoles[i][0], dipoles[i][1], head_width=0.2, head_length=0.2, fc='r', ec='r')
plt.title('Atom Positions and Dipole Moments')
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.grid(True)
plt.legend()
plt.axis('equal')
plt.show()
# Create a heatmap for state probabilities
reordered_indices = np.roll(np.arange(N), shift=-N//2)
y1 = y1[:, reordered_indices]
plt.figure(figsize=(10, 6))
plt.imshow(y1.T, aspect='auto', origin='lower', cmap='viridis', extent=[0, t_max, 0, N])
plt.colorbar(label='Probability')
plt.xlabel('Time')
plt.ylabel('Atom index - centered_index')
plt.title('Time evolution of atom state probabilities')
plt.show()

# Plot survival probability
plt.figure(figsize=(10, 6))
plt.plot(times, np.exp(-times), 'r--', linewidth=2.5, label=r'exp(-$\gamma t$)')
plt.plot(times, y2, 'b-', label='Survival Probability')
plt.xlabel('Time')
plt.ylabel('Survival Probability')
plt.title('Survival Probability over time')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()